In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import sys

sys.path.insert(0, "python")
from network import *
from protocols import *
from adversary import *
from simulator import *
from experiments import run_and_eval, run_experiment

In [ ]:
def shorten_protocol_name(x: str):
    val = x.replace("Protocol", "").replace("spreading_proba", "p")
    val = val.split("broadcast")[0][:-1].replace("(", ": ")
    if val[-1] == ",":
        val = val[:-1]
    return val

In [ ]:
nw_generator = NodeWeightGenerator("random")
ew_generator = EdgeWeightGenerator("normal")

rng = np.random.default_rng(32)
G = nx.random_regular_graph(3,10,seed=rng)
print(list(G.neighbors(0)))
nx.draw(G)

rng = np.random.default_rng(32)
rng.choice(G.nodes(), 3)

net = Network(nw_generator, ew_generator, num_nodes=1000, k=20)

seed = 41
net = Network(nw_generator, ew_generator, num_nodes=10, k=2, seed=seed)

np.mean(list(dict(net.graph.degree).values()))

tor = TOREnhancedProtocol(net, 1, 2, broadcast_mode="all", seed=seed)

np.mean(list(dict(net.graph.degree).values()))

tor.tor_network

msg = Message(0)
adv = Adversary(tor, 0.3, seed=seed)

adv.nodes

msg.queue

msg.process(tor, adv)

list(net.graph.neighbors(4))

# Dandelion example with random seed

In [ ]:
seed = 42

In [ ]:
net = Network(nw_generator, ew_generator, num_nodes=100, k=20, seed=seed)
dp = DandelionProtocol(net, 0.5, seed=seed)
adv = Adversary(dp, 0.1, seed=seed)
sim = Simulator(adv, 10, seed=seed, verbose=False)

In [ ]:
%%time
_ = sim.run()

In [ ]:
%%time
evaluator = Evaluator(sim, "first_reach")
print(evaluator.get_report())

In [ ]:
%%time
evaluator = Evaluator(sim, "first_sent")
print(evaluator.get_report())

In [ ]:
%%time
evaluator = Evaluator(sim, "dummy")
print(evaluator.get_report())

# Unittest in the making

In [ ]:
import sys, os, pytest

sys.path.insert(0, "%s/python" % os.getcwd())
import networkx as nx
from network import Network, NodeWeightGenerator, EdgeWeightGenerator
from simulator import Simulator, Evaluator
from message import Message
from protocols import BroadcastProtocol, DandelionProtocol, DandelionPlusPlusProtocol
from adversary import Adversary

In [ ]:
### First sent vs First reach test
rnd_node_weight = NodeWeightGenerator("random")
rnd_edge_weight = EdgeWeightGenerator("random")
G = nx.DiGraph()
G.add_nodes_from([1, 2, 3])
G.add_weighted_edges_from([(1, 2, 0.9), (1, 3, 1.84), (2, 3, 1.01)], weight="latency")
print(nx.draw(G, with_labels=True))
net = Network(rnd_node_weight, EdgeWeightGenerator("custom"), graph=G)
protocol = BroadcastProtocol(net, seed=44)
adv = Adversary(net, ratio=0.0, adversaries=[2])
print(adv.nodes)
assert 3 in adv.nodes

# start a message from Node 1
msg = Message(1)
receiver_order = [1, 2, 2, 2]
for i, receiver in enumerate(receiver_order):
    msg.process(protocol, adv)
    print(i, receiver, msg.history)
    assert receiver in msg.history
    # assert len(msg.history) == i + 1

In [ ]:
sim = Simulator(protocol, adv, 1, True)
sim.run(1.0)
for estimator in ["first_reach", "first_sent"]:
    evaluator = Evaluator(sim, estimator)
    results = [
        evaluator.exact_hits,
        evaluator.ranks,
        evaluator.inverse_ranks,
        evaluator.entropies,
    ]
    for i in results:
        print(estimator, i)

In [ ]:
rnd_node_weight = NodeWeightGenerator("random")
rnd_edge_weight = EdgeWeightGenerator("random")
G = nx.DiGraph()
G.add_nodes_from([1, 2, 3])
G.add_weighted_edges_from([(1, 2, 0.9), (1, 3, 1.84), (2, 3, 0.85)], weight="latency")

net = Network(rnd_node_weight, EdgeWeightGenerator("custom"), graph=G)
protocol = BroadcastProtocol(net, seed=44)
adv = Adversary(net, ratio=0.0, adversaries=[2])

assert 3 in adv.nodes

# start a message from Node 1
msg = Message(1)
receiver_order = [1, 2, 2, 2, 2]
for i, receiver in enumerate(receiver_order):
    msg.process(protocol, adv)
    assert receiver in msg.history

msg.flush_queue(adv)
print(msg.queue)
print("blbalbal", msg.mid)


sim = Simulator(
    protocol, Adversary(net, ratio=0.0, adversaries=[2]), 1, messages=[Message(1)]
)
# assert adv.predict_msg_source("first_sent")[msg.mid, G.nodes[1]] == 1
print("First", adv.predict_msg_source("first_sent"))
print("Cmon", adv.predict_msg_source("first_reach"))
print("Second", adv.predict_msg_source("first_reach").iloc[0][2])